In [111]:
print("chroma db")

chroma db


In [112]:
# !pip install chromadb-client==1.0.0
# !pip install --upgrade chromadb==1.0.0
#!pip install --upgrade langchain-chroma

In [113]:
# !pip install chromadb tiktoken transformers sentence_transformers pypdf

In [114]:
# !pip install langchain_community
# !pip install tiktoken

In [115]:
# !pip install sentence_transformers

In [116]:
# !pip install chromadb-client

In [117]:
# !pip install faiss-cpu

In [118]:
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_community.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain_community.vectorstores import Chroma # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_core.documents import Document # type: ignore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
import chromadb
from chromadb.config import Settings
import json
import uuid
import os, sys
import tiktoken
import warnings
warnings.filterwarnings("ignore")

In [119]:
os.environ['CURL_CA_BUNDLE'] = ''

In [120]:
print("curl http://localhost:8000/api/v1/heartbeat")

curl http://localhost:8000/api/v1/heartbeat


In [121]:
# Reference URL 
# https://www.jiniai.biz/2023/10/29/chroma-db-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B0%80%EC%9D%B4%EB%93%9C-python/

In [122]:
# -- chroma run
# chroma run --path /db_path
# chroma run --path ./Langchain/chromdb_run

In [123]:
"""
docs = [
            Document(page_content="사과", metadata=dict(page=1)),
            Document(page_content="애플", metadata=dict(page=1)),
            Document(page_content="바나나", metadata=dict(page=2)),
            Document(page_content="오렌지", metadata=dict(page=2)),
            Document(page_content="고양이", metadata=dict(page=3)),
            Document(page_content="야옹", metadata=dict(page=3)),
            Document(page_content="강아지", metadata=dict(page=4)),
            Document(page_content="멍멍", metadata=dict(page=4)),
            Document(page_content="해", metadata=dict(page=5)),
            Document(page_content="달", metadata=dict(page=5)),
            Document(page_content="물", metadata=dict(page=6)),
            Document(page_content="불", metadata=dict(page=6)),
            Document(page_content="apple", metadata=dict(page=7)),
]
"""

'\ndocs = [\n            Document(page_content="사과", metadata=dict(page=1)),\n            Document(page_content="애플", metadata=dict(page=1)),\n            Document(page_content="바나나", metadata=dict(page=2)),\n            Document(page_content="오렌지", metadata=dict(page=2)),\n            Document(page_content="고양이", metadata=dict(page=3)),\n            Document(page_content="야옹", metadata=dict(page=3)),\n            Document(page_content="강아지", metadata=dict(page=4)),\n            Document(page_content="멍멍", metadata=dict(page=4)),\n            Document(page_content="해", metadata=dict(page=5)),\n            Document(page_content="달", metadata=dict(page=5)),\n            Document(page_content="물", metadata=dict(page=6)),\n            Document(page_content="불", metadata=dict(page=6)),\n            Document(page_content="apple", metadata=dict(page=7)),\n]\n'

In [124]:
# -- local machine using the disk space
# chroma_client = chromadb.PersistentClient(path="./chromadb_data")
# -- docker chroma db
# settings = Settings(chroma_api_impl="chromadb.api.fastapi.FastAPI")
# chroma_client = chromadb.HttpClient(host="localhost", port=8000, settings=settings)
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [125]:
# embeddings = HuggingFaceEmbeddings(
#         model_name="BAAI/bge-m3",
#         model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
#         encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
#     )

In [126]:
# vector_store = Chroma(
#     collection_name="example_collection",
#     embedding_function=embeddings,
#     persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
# )

In [127]:
print("In Progressing..")

In Progressing..


In [128]:
chroma_client.heartbeat()

1748545833341528900

In [129]:
try:
    print(chroma_client.list_collections()) # collection 목록 가져오기
except Exception as e:
    print(str(e))

[Collection(name=new_collection)]


In [ ]:
try:
    collection = chroma_client.get_or_create_collection(name="new_collection")
    print(f"get_or_create_collection : {collection.peek()}")
    
#     collection = chroma_client.get_collection("new_collection")
#     print(collection.peek())
    
    print(f"chroma_client.list_collections() : {chroma_client.list_collections()}")

    collection_count = chroma_client.count_collections()
    print(f"collection_count : {collection_count}")

except Exception as e:
    print(e)

get_or_create_collection : {'ids': [], 'embeddings': array([], dtype=float64), 'metadatas': [], 'documents': [], 'data': None, 'uris': None, 'included': ['metadatas', 'documents', 'embeddings']}
chroma_client.list_collections() : [Collection(name=new_collection)]
collection_count : 1


In [ ]:
class MySentenceEmbedding():
    def __init__(self):
        # self.embeddings = HuggingFaceEmbeddings(
        #     model_name="BAAI/bge-m3",
        #     model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        #     encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
        # )
        self.default_embeddings = embedding_functions.DefaultEmbeddingFunction()
        
    def get_sentence_embedding(self, docs):
        # embed = self.embeddings.embed_documents(docs)
        embed = self.default_embeddings(docs)
        print(f"len(embed): {len(embed)}")
        # print(f"embed : {embed}")
        return embed

In [ ]:
rst = MySentenceEmbedding().get_sentence_embedding(["Love is wanting to be loved", "test"])

In [ ]:
# print(rst)
# sys.exit()

In [ ]:
documents=[
    "This is a document about pineapple",
    # "This is a document about oranges",
    # "This is a document about apples",
    # "I'm JeongTae Park"
]

In [ ]:
# embeddings = [
#     MySentenceEmbedding().get_sentence_embedding([document]) for document in documents
# ]
embeddings = MySentenceEmbedding().get_sentence_embedding(documents)

In [ ]:
# print(f"embeddings : {embeddings}")

In [ ]:
ids = [str(uuid.uuid4()) for i in range(len(documents))]

In [ ]:
print(f"ids : {ids}")

In [ ]:
metadatas = [
    {"index": i, "version": 1.0} for i in range(len(documents))
]

In [ ]:
print(f"metadata : {metadatas}")

In [ ]:
collection = chroma_client.get_collection("new_collection")
print(collection.peek())

In [ ]:
# Chroma will store your text and handle embedding and indexing automatically. 
# You can also customize the embedding model. You must provide unique string IDs for your documents.
# https://docs.trychroma.com/docs/overview/getting-started
collection.add(
    documents=documents,
    # embeddings=embeddings,
    # metadatas=metadatas,
    ids = ids
)

In [ ]:
collection = chroma_client.get_collection("new_collection")
print(collection.peek())

In [ ]:
print("Done..")